# ETL Transform

## Modules

In [1]:
# Importing the needed modules
import pandas as pd
import numpy as np
import plotly.figure_factory as ff
import scipy

## Preamble

Loading the raw CSV files as pandas dataframes.

In [2]:
# Loading the raw_data.csv and incremental_data.csv files into pandas dataframes
df_full = pd.read_csv("data/raw_data.csv")
df_incremental = pd.read_csv("data/incremental_data.csv")

## Helper Functions

In [3]:
# Saving to a CSV file
def save_to_csv(df, filename):
    """
    Write the pandas DataFrame to a CSV file.
    """
    df.to_csv(filename, index=False)

In [4]:
# Handling missing values using imputation
def handling_missing_values_with_viz(df, show_viz=True):
    """
    Impute missing values with optional visualizations.
    """
    df = df.copy()
    figures = {}

    for col in df.columns:
        series = df[col]
        if series.isnull().any():
            if pd.api.types.is_numeric_dtype(series):
                clean = series.dropna()
                is_int_col = clean.map(float.is_integer).all()
                skewness = clean.skew()
                fill_val = clean.mean() if abs(skewness) < 0.5 else clean.median()
                
                if is_int_col:
                    fill_val = int(round(fill_val))
                
                df[col] = series.fillna(fill_val)

                if is_int_col:
                    df[col] = df[col].astype('Int64')
            else:
                modes = series.mode()
                if not modes.empty:
                    df[col] = series.fillna(modes[0])

        # Plot numeric columns
        if show_viz and pd.api.types.is_numeric_dtype(df[col]):
            clean2 = df[col].dropna()
            fig = ff.create_distplot([clean2], [col],
                                     show_hist=True, show_rug=True, show_curve=True)
            fig.update_layout(title=f"{col} distribution (skewness={clean2.skew():.2f})")
            figures[col] = fig

    return (df, figures) if show_viz else df

In [5]:
# Handling duplicate values
def handle_duplicates(df):
    """
    Locate and drop duplicate rows.
    """
    df = df.drop_duplicates().copy()
    return df

In [6]:
# Dropping irrelevant columns
def drop_irrelevant_columns(df, columns):
    """
    Drop specified columns from the DataFrame.
    """
    df = df.drop(columns=columns, errors='ignore').copy()
    return df

In [7]:
# Converting the Date Data Type into Text
def format_date_column(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col] = df[col].dt.strftime('%d %B %Y')
    return df

## Data Observation

In [8]:
# Observe the raw data CSV file
df_full.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1,Diana,Tablet,NaN,500.0,2024-01-20,South
1,2,Eve,Laptop,NaN,NaN,2024-04-29,North
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,NaN
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West
4,5,Eve,Tablet,3.0,NaN,2024-03-07,South


In [9]:
# Observe the incremental data CSV file
df_incremental.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,101,Alice,Laptop,NaN,900.0,2024-05-09,Central
1,102,NaN,Laptop,1.0,300.0,2024-05-07,Central
2,103,NaN,Laptop,1.0,600.0,2024-05-04,Central
3,104,NaN,Tablet,NaN,300.0,2024-05-26,Central
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North


## Data Cleaning

As noted in the ETL Extract Jupyter Notebook the `raw_data.csv` file has:

- Null values
- Duplicate rows

The `incremental_data.csv` file has:

- Null values

Both of them have:

- The `order_id` column that should be omitted.

### Raw Data

#### Irrelevant Columns

Before irrelevant columns are handled:

In [10]:
# Initial state before handling irrelevant columns
df_full.head(2)

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1,Diana,Tablet,NaN,500.0,2024-01-20,South
1,2,Eve,Laptop,NaN,NaN,2024-04-29,North


To handle irrelevant columns the `order_id` column will be dropped. This column offers no insights when performing data analysis.

In [11]:
# Handling the irrelevant column
df_full = drop_irrelevant_columns(df_full, ["order_id"])

After handling the irrelevant column:

In [12]:
# After handling the irrelevant column
df_full.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Diana,Tablet,NaN,500.0,2024-01-20,South
1,Eve,Laptop,NaN,NaN,2024-04-29,North


#### Null Values

Before null values are handled:

In [13]:
# Initial state before handling null values
df_full_null_counts = df_full.isnull().sum()
print(df_full_null_counts)

customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64


To handle null values a selection between:

- Mean imputation
- Median imputation
- Mode imputation

will be done dependent on the data type and/or distribution.

This will ensure that rows with null values will not need to be dropped. Rather, they will align to the data distribution of the pre-existing data.

In [14]:
# Imputing null values with graphing to check skewness
df_full, dist_full_figs = handling_missing_values_with_viz(df_full)

In [15]:
# Displaying the graphs showing the skewness
for col, fig in dist_full_figs.items():
    print(f"Plot for '{col}':")
    fig.show()

Plot for 'quantity':


Plot for 'unit_price':


After null values are handled:

In [16]:
# State after handling null values
df_full_null_counts = df_full.isnull().sum()
print(df_full_null_counts)

customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64


#### Duplicate Rows

Before duplicate rows were handled:

In [17]:
# Initial state before handling duplicate rows
df_full_num_dupes = df_full.duplicated().sum()
print(df_full_num_dupes)

1


To handle duplicate rows, the duplicate rows will be dropped. This ensures that there is no redundancy in the data set.

In [18]:
# Deleting duplicate rows
df_full = handle_duplicates(df_full)

After duplicate rows are handled:

In [19]:
# After deleting duplicate rows
df_full_num_dupes = df_full.duplicated().sum()
print(df_full_num_dupes)

0


### Incremental Data

#### Irrelevant Columns

Before irrelevant columns are handled:

In [20]:
# Initial state before handling irrelevant columns
df_incremental.head(2)

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,101,Alice,Laptop,NaN,900.0,2024-05-09,Central
1,102,NaN,Laptop,1.0,300.0,2024-05-07,Central


To handle irrelevant columns the `order_id` column will be dropped. This column offers no insights when performing data analysis.

In [21]:
# Handling the irrelevant column
df_incremental = drop_irrelevant_columns(df_incremental, ["order_id"])

After handling the irrelevant column:

In [22]:
# After handling the irrelevant column
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Alice,Laptop,NaN,900.0,2024-05-09,Central
1,NaN,Laptop,1.0,300.0,2024-05-07,Central


#### Null Values

Before null values were handled:

In [23]:
# Initial state before handling irrelevant columns
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Alice,Laptop,NaN,900.0,2024-05-09,Central
1,NaN,Laptop,1.0,300.0,2024-05-07,Central


To handle null values a selection between:

- Mean imputation
- Median imputation
- Mode imputation

will be done dependent on the data type and/or distribution.

This will ensure that rows with null values will not need to be dropped. Rather, they will align to the data distribution of the pre-existing data.

In [24]:
# Imputing null values with graphing to check skewness
df_incremental, dist_incremental_figs = handling_missing_values_with_viz(df_incremental)

In [25]:
# Displaying the graphs showing the skewness
for col, fig in dist_incremental_figs.items():
    print(f"Plot for '{col}':")
    fig.show()

Plot for 'quantity':


Plot for 'unit_price':


After null values are handled:

In [26]:
# State after handling null values
df_incremental_null_counts = df_incremental.isnull().sum()
print(df_incremental_null_counts)

customer_name    0
product          0
quantity         0
unit_price       0
order_date       0
region           0
dtype: int64


## Data Enrichment

Both the Raw Data and Incremental Data will have 

### Data Type Conversion

Before the conversion:

In [27]:
# Displaying the initial dataframes
df_full.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Diana,Tablet,2,500,2024-01-20,South
1,Eve,Laptop,2,500,2024-04-29,North


In [28]:
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Alice,Laptop,2,900.0,2024-05-09,Central
1,Heidi,Laptop,1,300.0,2024-05-07,Central


The `order_date` column changed from datetime to text in the format of `<Date in Numbers> <Month in Words> <Year in Numbers>`. This format is human readable for ease of data analysis.

In [29]:
# Converting the order_date column
df_full = format_date_column(df_full, "order_date")
df_incremental = format_date_column(df_incremental, "order_date")

After the conversion:

In [30]:
# Displaying the enriched dataframe
df_full.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Diana,Tablet,2,500,20 January 2024,South
1,Eve,Laptop,2,500,29 April 2024,North


In [31]:
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Alice,Laptop,2,900.0,09 May 2024,Central
1,Heidi,Laptop,1,300.0,07 May 2024,Central


### Summary Metrics

Before the summary metrics:

In [32]:
# Displaying the initial dataframes
df_full.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Diana,Tablet,2,500,20 January 2024,South
1,Eve,Laptop,2,500,29 April 2024,North


In [33]:
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region
0,Alice,Laptop,2,900.0,09 May 2024,Central
1,Heidi,Laptop,1,300.0,07 May 2024,Central


The `quantity` and `unit_price` columns are multiplied to create the `total_price` column. This column offers a summary statistic of the total amount in cost that a particular sale cost.

In [34]:
# Creating a total_price column formed by quantity * unit_price
df_full['total_price'] = df_full['quantity'] * df_full['unit_price']
df_incremental['total_price'] = df_incremental['quantity'] * df_incremental['unit_price']

After the summary metrics:

In [35]:
# Displaying the enriched dataframe
df_full.head(2)

,customer_name,product,quantity,unit_price,order_date,region,total_price
0,Diana,Tablet,2,500,20 January 2024,South,1000
1,Eve,Laptop,2,500,29 April 2024,North,1000


In [36]:
df_incremental.head(2)

,customer_name,product,quantity,unit_price,order_date,region,total_price
0,Alice,Laptop,2,900.0,09 May 2024,Central,1800.0
1,Heidi,Laptop,1,300.0,07 May 2024,Central,300.0


## Data Saving

In [37]:
# Saving the cleaned and enriched dataframes to the transformed folder
save_to_csv(df_full, "transformed/transformed_full.csv")
save_to_csv(df_incremental, "transformed/transformed_incremental.csv")